<a href="https://colab.research.google.com/github/sitori8354/eatago/blob/main/Eatago.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -U pywebio
# !pip install -U https://github.com/solrz/PyWebIO/archive/dev.zip

Requirement already up-to-date: pywebio in /usr/local/lib/python3.7/dist-packages (1.2.3)


In [2]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!wget https://raw.githubusercontent.com/solrz/pywebio-example/main/hello_world.py

!pip3 install nest-asyncio

--2021-05-14 18:44:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.227.65.201, 52.204.244.158, 52.204.93.39, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.227.65.201|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.6’

ngrok-stable-linux- 100%[===================>]  13.19M  55.9MB/s    in 0.2s    

2021-05-14 18:44:18 (55.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.6’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
--2021-05-14 18:44:23--  https://raw.githubusercontent.com/solrz/pywebio-example/main/hello_world.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com

In [3]:
import random
import os
import time
assign_port = 80
import nest_asyncio
from pywebio.input import *
from pywebio.output import *
import pywebio
from multiprocessing import Process
import sys, json
import asyncio

try:
  del new_server
  del start_app
except:
  pass

nest_asyncio.apply()
def new_server(target_app, port):
  try:
    if pywebio:
      from importlib import reload
      reload(pywebio)
  except:
    pass
  return lambda: pywebio.start_server(target_app, port=port)

previous_process = None
ports = [80]

def start_app(target_app):
  assign_port = random.randint(5000,9999)
  global ports
  while assign_port in ports:
    assign_port += 1
  ports += [assign_port]
  tunnel_port = f'1{assign_port}'
  ngrok_file = f'./ngrok_config_{tunnel_port}'
  with open(ngrok_file, 'w') as f:
    f.write('web_addr: '+tunnel_port)
  get_ipython().system_raw(f'./ngrok http {assign_port} --config "{ngrok_file}" &')
  time.sleep(2)
  forward_info_raw = !curl -s http://localhost:$tunnel_port/api/tunnels 
  # print(forward_info_raw)

  forward_info = json.loads(forward_info_raw[0])
  print(f'請拖曳網址到新視窗來打開App（每次網址都會更新喔！）: \n{forward_info["tunnels"][0]["public_url"]}')
  # | python3 -c \
  #    'import sys, json; print("請拖曳網址到新視窗來打開App（每次網址都會更新喔！）: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'
  global previous_process
  if previous_process:
    previous_process.terminate()
  previous_process = Process(target=new_server(target_app, assign_port))
  previous_process.daemon = True
  previous_process.start()



In [4]:
def example_hello_world_app():
  put_text('hello_world！')
  name = input('花枝魷魚麵？')
  put_text(f'{name} 早安, 歡迎使用我的App')

def inertactive_app():
  enable_print_convert_to_put_text = True
  put_markdown('# Python 即時編譯Web App')
  put_markdown('在這裡，你可以直接輸入程式碼，快速測試你的code')
  code = "put_markdown('# 我可以讀透你的內心...')\nname = input('首先，請問大名？')\nput_text(f'你的名字叫{name}，猜對了吧！')\n"
  round = 1
  while True:
    put_text(f'第{round}次執行')
    code = textarea('輸入程式', code={
      'mode': "python",  # code language
      'theme': 'darcula',  # Codemirror theme. Visit https://codemirror.net/demo/theme.html#cobalt to get more themes
    }, value=code)
    put_markdown('---')
    put_code(code)
    try:
      exec(code.replace('print(','put_text('))
    except Exception as e:
      put_text(e)
    put_markdown('---')
    round += 1


# 把 start_app(example_hello_world_app) 的 「example_hello_world_app」是會執行App


In [5]:
def task_1():
    put_text('task_1')
    put_buttons(['Go task 2'], [lambda: go_app('task_2')])
    hold()

def task_2():
    put_text('task_2')
    put_buttons(['Go task 1'], [lambda: go_app('task_1')])
    gift = select('what?', ['ta','ya'])
    hold()

def index():
    put_link('Go task 1', app='task_1')  #  使用app参数指定任务名
    put_link('Go task 2', app='task_2')

# 等价于 start_server({'index': index, 'task_1': task_1, 'task_2': task_2})
start_app([index, task_1, task_2])

請拖曳網址到新視窗來打開App（每次網址都會更新喔！）: 
https://950ebb9c7872.ngrok.io
Listen on 0.0.0.0:7606
